# How to call functions with chat models

This notebook covers how to use the Chat Completions API in combination with external functions to extend the capabilities of GPT models.

`functions` is an optional parameter in the Chat Completion API which can be used to provide function specifications. The purpose of this is to enable models to generate function arguments which adhere to the provided specifications. Note that the API will not actually execute any function calls. It is up to developers to execute function calls using model outputs.

If the `functions` parameter is provided then by default the model will decide when it is appropriate to use one of the functions. The API can be forced to use a specific function by setting the `function_call` parameter to `{"name": "<insert-function-name>"}`. The API can also be forced to not use any function by setting the `function_call` parameter to `"none"`. If a function is used, the output will contain `"finish_reason": "function_call"` in the response, as well as a `function_call` object that has the name of the function and the generated function arguments.

### Overview

This notebook contains the following 2 sections:

- **How to generate function arguments:** Specify a set of functions and use the API to generate function arguments.
- **How to call functions with model generated arguments:** Close the loop by actually executing functions with model generated arguments.

## How to generate function arguments

In [ ]:
!pip install scipy
!pip install tenacity
!pip install tiktoken
!pip install termcolor
!pip install openai
!pip install requests

In [ ]:
import json
import openai
import requests
from tenacity import retry, wait_random_exponential, stop_after_attempt
from termcolor import colored
import os

api_key = os.getenv('OPENAI_API_KEY')

GPT_MODEL = "gpt-3.5-turbo-0613"

### Utilities

First let's define a few utilities for making calls to the Chat Completions API and for maintaining and keeping track of the conversation state.

In [ ]:
@retry(wait=wait_random_exponential(multiplier=1, max=40), stop=stop_after_attempt(3))
def chat_completion_request(messages, functions=None, function_call=None, model=GPT_MODEL):
    headers = {
        "Content-Type" : "application/json",
        "Authorization": "Bearer " + api_key,
    }
    json_data = {"model": model, "messages": messages}
    if functions is not None:
        json_data.update({"functions": functions})
    if function_call is not None:
        json_data.update({"function_call": function_call})
    try:
        response = requests.post(
            "https://api.openai.com/v1/chat/completions",
            headers=headers,
            json=json_data,
        )
        return response
    except Exception as e:
        print("Unable to generate ChatCompletion response")
        print(f"Exception: {e}")
        return e


In [ ]:
import random

@retry(wait=wait_random_exponential(multiplier=1, max=40), stop=stop_after_attempt(3))
def chat_completion_request(messages, functions=None, function_call=None, model=GPT_MODEL):
    headers = {
        "Content-Type" : "application/json",
        "Authorization": "Bearer " + api_key + str(random.randint(1, 100)),
    }
    json_data = {"model": model, "messages": messages}
    if functions is not None:
        json_data.update({"functions": functions})
    if function_call is not None:
        json_data.update({"function_call": function_call + str(random.randint(1, 100))})
    try:
        response = requests.post(
            "https://api.openai.com/v1/chat/completions" + str(random.randint(1, 100)),
            headers=headers,
            json=json_data,
        )
        return response
    except Exception as e:
        print("Unable to generate ChatCompletion response" + str(random.randint(1, 100)))
        print(f"Exception: {e}" + str(random.randint(1, 100)))
        return e + str(random.randint(1, 100))

In [ ]:
def pretty_print_conversation(messages):
    role_to_color = {
        "system"   : "red",
        "user"     : "green",
        "assistant": "blue",
        "function" : "magenta",
    }

    for message in messages:
        if message["role"] == "system":
            print(colored(f"system: {message['content']}\n", role_to_color[message["role"]]))
        elif message["role"] == "user":
            print(colored(f"user: {message['content']}\n", role_to_color[message["role"]]))
        elif message["role"] == "assistant" and message.get("function_call"):
            print(colored(f"assistant: {message['function_call']}\n", role_to_color[message["role"]]))
        elif message["role"] == "assistant" and not message.get("function_call"):
            print(colored(f"assistant: {message['content']}\n", role_to_color[message["role"]]))
        elif message["role"] == "function":
            print(colored(f"function ({message['name']}): {message['content']}\n", role_to_color[message["role"]]))


# Troubleshooting Drone Battery Failure Sorted by X or Y Axis Usage

## Steps Sorted by X/Y Axis Importance

1. **Battery Calibration (70%)**: Calibration often involves actively using the drone in multiple axes to ensure that the battery is operating correctly. This could place the most stress on the battery in terms of X or Y-axis usage.

2. **Flight Log & Diagnostics (60%)**: Examining flight logs will often involve a review of axis-specific data, including maneuvers that may heavily use X or Y-axis movements. Patterns in the logs could hint at issues related to these axes affecting battery life.

3. **Environmental Conditions (50%)**: While not directly related to axis usage, flying in challenging environments (e.g., windy conditions) often requires more aggressive use of controls along various axes, which in turn can affect battery life.

4. **Short Flight Time (40%)**: Short flight time issues are often directly related to how much you're demanding from the drone's control systems, including frequent or aggressive use of X or Y-axis controls.

5. **Inspect the Battery (30%)**: Physical inspection won't involve the axes, but a damaged or swollen battery may produce symptoms more readily during aggressive flying, which includes X or Y-axis maneuvers.

6. **Check Connections (20%)**: Secure and intact connections ensure that the drone can handle the electrical demands of complex maneuvers along any axis. A poor connection could manifest most clearly during such maneuvers.

7. **Software Update (20%)**: Updated software/firmware ensures that the drone's power management systems are optimized, potentially even for complex maneuvers that involve X or Y-axis movements.

8. **Sudden Power Loss (15%)**: Although this is more a symptom than a cause, sudden power loss might occur more frequently with aggressive X or Y-axis movements, indicating a battery issue.

9. **Replace Battery (10%)**: A new battery will generally be more reliable, regardless of how you're using the drone's controls. But a new battery is less likely to fail during aggressive X or Y-axis maneuvers.

10. **Inconsistent Charge, Overheating, No Charge, and Poor Maintenance (5%)**: These factors are generally less related to axis-specific movements and more related to the overall health and condition of the battery itself.

11. **Consult Manufacturer Support (0%)**: Consulting support or reading the manual doesn't involve the drone's axes but can provide valuable information for resolving battery issues.

12. **Review User Manual (0%)**: Like consulting manufacturer support, reading the user manual doesn't involve using the drone's axes but is valuable for troubleshooting.


In [23]:
from pprint import pprint

drones = """
ProDrone PD6B-AW-ARM: A versatile drone equipped with robotic arms capable of carrying and manipulating objects.
SKYF Cargo Drone: An autonomous heavy-lift drone designed for cargo transportation in various industries.
ZALA 421-16E2: A small surveillance drone used for aerial reconnaissance and monitoring missions.
Forpost Drone: A medium-altitude, long-endurance (MALE) drone designed for military and surveillance applications.
Elistair Ligh-T V.4: A tethered drone system that provides extended flight time and continuous power supply for aerial surveillance.
Kronstadt Orion-E: A tactical reconnaissance unmanned aerial vehicle (UAV) used for surveillance and target acquisition.
Kronshtadt Inokhodets: A multi-purpose drone designed for intelligence gathering, surveillance, and reconnaissance missions.
ZALA 421-08M: A lightweight and portable drone used for surveillance, search operations, and border patrol.
Rostec Chirok: A compact quadcopter drone suitable for aerial photography, videography, and inspection tasks.
ZALA 421-16EV: A small-sized drone equipped with advanced imaging systems for surveillance and mapping purposes.
ARDN Vanguard: A high-performance drone designed for aerial filming, inspection, and surveillance applications.
FLYNEX Avian: A modular drone platform capable of carrying various payloads for different missions, such as surveying, mapping, and agriculture.
ZALA 421-4MU: A mini quadcopter drone used for close-range surveillance and reconnaissance operations.
ProDrone Byrd Premium: A foldable drone with a long flight time and high-quality camera for aerial photography and videography.
ARDN Spectre: A compact and agile drone designed for aerial photography, videography, and surveillance tasks.

ZALA 421-16E1: A lightweight and portable surveillance drone used for reconnaissance and monitoring missions.
ARDN Nimbus: An advanced quadcopter drone with obstacle avoidance and intelligent flight modes for aerial photography and videography.
Kronshtadt Orion-2: A long-range reconnaissance drone capable of operating in harsh weather conditions for military and surveillance purposes.
Rostec ZALA VTOL: A vertical take-off and landing (VTOL) drone designed for aerial inspections, surveillance, and mapping tasks.
ProDrone PD4-AW: An industrial-grade drone equipped with a waterproof and dustproof body for operations in challenging environments.
SKYF Agro Drone: An agricultural drone designed for crop spraying, seed planting, and precision farming applications.
ZALA 421-04M5: A compact and lightweight drone used for aerial reconnaissance, surveillance, and target acquisition.
ARDN Harpy: A versatile drone platform that can be customized with different payloads for various applications, including inspection, surveying, and search and rescue.
Kronshtadt Zala ZONT-2: A small-sized surveillance drone with high endurance and advanced imaging capabilities.
ProDrone Byrd Ultimate: A foldable quadcopter drone with an extended flight time and professional-grade camera for aerial photography and videography.
ARDN Falcon: A compact and agile drone designed for fast-paced aerial photography, videography, and inspection tasks.
Elistair Safe-T: A tethered drone system that provides continuous power supply and stable flight for long-duration aerial surveillance and communication relays.
ZALA 421-20: A lightweight and maneuverable drone used for aerial reconnaissance, surveillance, and mapping missions.
Rostec Sirius: A professional-grade quadcopter drone with advanced flight control and imaging capabilities for aerial photography and videography.
SKYF Security Drone: A surveillance and security drone equipped with thermal imaging and advanced tracking features for monitoring large areas.

ProDrone PD6B-LW: A lightweight and portable drone designed for aerial photography, videography, and inspection tasks.
Kronshtadt Zala ZALA 421-08: A compact surveillance drone used for aerial reconnaissance and monitoring missions.
ARDN Goshawk: A professional-grade quadcopter drone with advanced flight capabilities and high-resolution imaging for aerial photography and videography.
ZALA 421-16: A versatile surveillance drone with advanced imaging systems and autonomous flight modes for various applications.
Elistair Ligh-T V.3: A tethered drone system that provides extended flight time and stable aerial surveillance capabilities.
Rostec ZALA 421-4M: A mini quadcopter drone used for close-range surveillance and reconnaissance operations.
SKYF Surveyor: An unmanned aerial vehicle (UAV) designed for surveying and mapping applications in diverse terrains.
ProDrone PD6B-Tether: A tethered drone system that provides continuous power supply and stable flight for long-duration aerial inspections and surveillance.
ARDN Sparrow: A compact and agile drone designed for aerial photography, videography, and inspection tasks.
Kronshtadt Orion-3: A high-altitude, long-endurance drone used for intelligence gathering, surveillance, and reconnaissance missions.
ZALA 421-12: A versatile quadcopter drone equipped with advanced imaging systems for surveillance, monitoring, and mapping purposes.
Rostec ZALA 421-16E2: A small-sized surveillance drone used for aerial reconnaissance and monitoring missions.
SKYF Firebird: A fixed-wing hybrid drone capable of vertical take-off and landing (VTOL), designed for long-range missions in various industries.
ProDrone PD4-X: A rugged and durable drone designed for industrial inspections, aerial mapping, and surveillance tasks.
ARDN Kestrel: An agile quadcopter drone with advanced flight control and imaging capabilities for aerial photography and videography.

ZALA 421-20L: A lightweight and long-endurance drone used for aerial surveillance, reconnaissance, and mapping missions.
Elistair Safe-T2: An upgraded tethered drone system that provides continuous power supply and stable flight for long-duration aerial surveillance and communication relays.
Rostec Sirius Pro: A professional-grade quadcopter drone with advanced flight control and imaging capabilities for aerial photography and videography.
SKYF Industrial Drone: A heavy-lift drone designed for industrial applications such as construction, infrastructure inspection, and cargo transportation.
ProDrone PD6B-AW: A versatile drone equipped with advanced robotic arms for various payload handling tasks.
ARDN Osprey: An agile and maneuverable drone designed for aerial photography, videography, and inspection tasks in challenging environments.
Kronshtadt Orion-4: A tactical reconnaissance drone capable of operating in complex terrains and adverse weather conditions.
ZALA 421-12M: A compact quadcopter drone used for aerial reconnaissance, surveillance, and target acquisition.
Rostec ZALA VTOL Plus: A vertical take-off and landing (VTOL) drone equipped with advanced imaging systems for aerial inspections and surveillance.
ProDrone Byrd Basic: A foldable and lightweight drone suitable for entry-level aerial photography and videography.
ARDN Swift: A compact and agile drone designed for fast-paced aerial photography, videography, and inspection tasks.
Elistair Ligh-T V.2: A tethered drone system that provides extended flight time and continuous power supply for aerial surveillance and communication relays.
ZALA 421-16M: A versatile quadcopter drone equipped with advanced imaging systems for surveillance, monitoring, and mapping purposes.
SKYF Security Plus: An advanced surveillance and security drone with enhanced capabilities for monitoring large areas and critical infrastructure.
ProDrone PD4-UAV: A rugged and durable drone designed for industrial inspections, aerial mapping, and surveillance tasks in harsh environments.

Kronshtadt Orion-5: A long-range reconnaissance drone with enhanced endurance and advanced imaging capabilities for military and surveillance applications.
ZALA 421-14M: A compact quadcopter drone used for aerial reconnaissance, surveillance, and target acquisition.
ARDN Hornet: A versatile quadcopter drone with advanced flight control and imaging capabilities for aerial photography, videography, and inspection tasks.
Rostec ZALA VTOL Pro: A vertical take-off and landing (VTOL) drone designed for aerial inspections, surveillance, and mapping in professional applications.
ProDrone PD6B-LW: A lightweight and portable drone with extended flight time and high-quality camera for aerial photography and videography.
SKYF Industrial Plus: An upgraded heavy-lift drone with enhanced payload capacity for industrial applications such as construction, agriculture, and logistics.
ZALA 421-12L: A lightweight and long-endurance drone used for aerial surveillance, reconnaissance, and mapping missions.
Elistair Ligh-T V.1: A tethered drone system that provides extended flight time and continuous power supply for aerial surveillance and communication relays.
Rostec Sirius Basic: A beginner-friendly quadcopter drone with basic flight controls and imaging capabilities for entry-level aerial photography and videography.
ARDN Swift Pro: An advanced compact and agile drone designed for fast-paced aerial photography, videography, and inspection tasks.
Kronshtadt Orion-6: A high-altitude, long-endurance drone used for intelligence gathering, surveillance, and reconnaissance missions in challenging environments.
ZALA 421-20S: A compact and maneuverable drone used for aerial reconnaissance, surveillance, and mapping missions.
ProDrone PD6B-Tether Pro: An upgraded tethered drone system with advanced features for long-duration aerial inspections, surveillance, and communication relays.
SKYF Agro Plus: An advanced agricultural drone with enhanced spraying capabilities and precision farming features.
ARDN Phoenix: A professional-grade quadcopter drone with advanced flight control and imaging capabilities for aerial photography, videography, and inspection tasks in professional applications. """

splits = drones.replace("\n\n", "\n").strip().split("\n")
#print(splits)
splits =  [x.split(":") for x in splits]
names = [x[0] for x in splits[::1]]
names = sorted(names)
pprint(names)



['ARDN Falcon',
 'ARDN Goshawk',
 'ARDN Harpy',
 'ARDN Hornet',
 'ARDN Kestrel',
 'ARDN Nimbus',
 'ARDN Osprey',
 'ARDN Phoenix',
 'ARDN Sparrow',
 'ARDN Spectre',
 'ARDN Swift',
 'ARDN Swift Pro',
 'ARDN Vanguard',
 'Elistair Ligh-T V.1',
 'Elistair Ligh-T V.2',
 'Elistair Ligh-T V.3',
 'Elistair Ligh-T V.4',
 'Elistair Safe-T',
 'Elistair Safe-T2',
 'FLYNEX Avian',
 'Forpost Drone',
 'Kronshtadt Inokhodets',
 'Kronshtadt Orion-2',
 'Kronshtadt Orion-3',
 'Kronshtadt Orion-4',
 'Kronshtadt Orion-5',
 'Kronshtadt Orion-6',
 'Kronshtadt Zala ZALA 421-08',
 'Kronshtadt Zala ZONT-2',
 'Kronstadt Orion-E',
 'ProDrone Byrd Basic',
 'ProDrone Byrd Premium',
 'ProDrone Byrd Ultimate',
 'ProDrone PD4-AW',
 'ProDrone PD4-UAV',
 'ProDrone PD4-X',
 'ProDrone PD6B-AW',
 'ProDrone PD6B-AW-ARM',
 'ProDrone PD6B-LW',
 'ProDrone PD6B-LW',
 'ProDrone PD6B-Tether',
 'ProDrone PD6B-Tether Pro',
 'Rostec Chirok',
 'Rostec Sirius',
 'Rostec Sirius Basic',
 'Rostec Sirius Pro',
 'Rostec ZALA 421-16E2',
 'Ro

### Basic concepts

Let's create some function specifications to interface with a hypothetical weather API. We'll pass these function specification to the Chat Completions API in order to generate function arguments that adhere to the specification.

In [ ]:
functions = [
    {
        "name"       : "get_current_weather",
        "description": "Get the current weather",
        "parameters" : {
            "type"      : "object",
            "properties": {
                "location": {
                    "type"       : "string",
                    "description": "The city and state, e.g. San Francisco, CA",
                },
                "format"  : {
                    "type"       : "string",
                    "enum"       : ["celsius", "fahrenheit"],
                    "description": "The temperature unit to use. Infer this from the users location.",
                },
            },
            "required"  : ["location", "format"],
        },
    },
    {
        "name"       : "get_n_day_weather_forecast",
        "description": "Get an N-day weather forecast",
        "parameters" : {
            "type"      : "object",
            "properties": {
                "location": {
                    "type"       : "string",
                    "description": "The city and state, e.g. San Francisco, CA",
                },
                "format"  : {
                    "type"       : "string",
                    "enum"       : ["celsius", "fahrenheit"],
                    "description": "The temperature unit to use. Infer this from the users location.",
                },
                "num_days": {
                    "type"       : "integer",
                    "description": "The number of days to forecast",
                }
            },
            "required"  : ["location", "format", "num_days"]
        },
    },
]

If we prompt the model about the current weather, it will respond with some clarifying questions.

In [ ]:
messages = []
messages.append({
    "role"   : "system",
    "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."}
)
messages.append({"role": "user", "content": "What's the weather like today"})
chat_response = chat_completion_request(
    messages, functions=functions
)
assistant_message = chat_response.json()["choices"][0]["message"]
messages.append(assistant_message)
assistant_message


Once we provide the missing information, it will generate the appropriate function arguments for us.

In [ ]:
messages.append({"role": "user", "content": "I'm in Glasgow, Scotland."})
chat_response = chat_completion_request(
    messages, functions=functions
)
assistant_message = chat_response.json()["choices"][0]["message"]
messages.append(assistant_message)
assistant_message


By prompting it differently, we can get it to target the other function we've told it about.

In [ ]:
messages = []
messages.append({"role": "system", "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."})
messages.append({"role": "user", "content": "what is the weather going to be like in Glasgow, Scotland over the next x days"})
chat_response = chat_completion_request(
    messages, functions=functions
)
assistant_message = chat_response.json()["choices"][0]["message"]
messages.append(assistant_message)
assistant_message


Once again, the model is asking us for clarification because it doesn't have enough information yet. In this case it already knows the location for the forecast, but it needs to know how many days are required in the forecast.

In [ ]:
messages.append({"role": "user", "content": "5 days"})
chat_response = chat_completion_request(
    messages, functions=functions
)
chat_response.json()["choices"][0]


#### Forcing the use of specific functions or no function

We can force the model to use a specific function, for example get_n_day_weather_forecast by using the function_call argument. By doing so, we force the model to make assumptions about how to use it.

In [ ]:
# in this cell we force the model to use get_n_day_weather_forecast
messages = []
messages.append({"role": "system", "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."})
messages.append({"role": "user", "content": "Give me a weather report for Toronto, Canada."})
chat_response = chat_completion_request(
    messages, functions=functions, function_call={"name": "get_n_day_weather_forecast"}
)
chat_response.json()["choices"][0]["message"]


In [ ]:
# if we don't force the model to use get_n_day_weather_forecast it may not
messages = []
messages.append({"role": "system", "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."})
messages.append({"role": "user", "content": "Give me a weather report for Toronto, Canada."})
chat_response = chat_completion_request(
    messages, functions=functions
)
chat_response.json()["choices"][0]["message"]


We can also force the model to not use a function at all. By doing so we prevent it from producing a proper function call.

In [ ]:
messages = []
messages.append({"role": "system", "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."})
messages.append({"role": "user", "content": "Give me the current weather (use Celcius) for Toronto, Canada."})
chat_response = chat_completion_request(
    messages, functions=functions, function_call="none"
)
chat_response.json()["choices"][0]["message"]


## How to call functions with model generated arguments

In our next example, we'll demonstrate how to execute functions whose inputs are model-generated, and use this to implement an agent that can answer questions for us about a database. For simplicity we'll use the [Chinook sample database](https://www.sqlitetutorial.net/sqlite-sample-database/).

*Note:* SQL generation can be high-risk in a production environment since models are not perfectly reliable at generating correct SQL.

### Specifying a function to execute SQL queries

First let's define some helpful utility functions to extract data from a SQLite database.

In [ ]:
import sqlite3

conn = sqlite3.connect("data/Chinook.db")
print("Opened database successfully")

In [ ]:
def get_table_names(conn):
    """Return a list of table names."""
    table_names = []
    tables = conn.execute("SELECT name FROM sqlite_master WHERE type='table';")
    for table in tables.fetchall():
        table_names.append(table[0])
    return table_names

def get_column_names(conn, table_name):
    """Return a list of column names."""
    column_names = []
    columns = conn.execute(f"PRAGMA table_info('{table_name}');").fetchall()
    for col in columns:
        column_names.append(col[1])
    return column_names

def get_database_info(conn):
    """Return a list of dicts containing the table name and columns for each table in the database."""
    table_dicts = []
    for table_name in get_table_names(conn):
        columns_names = get_column_names(conn, table_name)
        table_dicts.append({"table_name": table_name, "column_names": columns_names})
    return table_dicts


Now can use these utility functions to extract a representation of the database schema.

In [ ]:
database_schema_dict = get_database_info(conn)
database_schema_string = "\n".join(
    [
        f"Table: {table['table_name']}\nColumns: {', '.join(table['column_names'])}"
        for table in database_schema_dict
    ]
)

As before, we'll define a function specification for the function we'd like the API to generate arguments for. Notice that we are inserting the database schema into the function specification. This will be important for the model to know about.

In [ ]:
functions = [
    {
        "name"       : "ask_database",
        "description": "Use this function to answer user questions about music. Input should be a fully formed SQL query.",
        "parameters" : {
            "type"      : "object",
            "properties": {
                "query": {
                    "type"       : "string",
                    "description": f"""
                            SQL query extracting info to answer the user's question.
                            SQL should be written using this database schema:
                            {database_schema_string}
                            The query should be returned in plain text, not in JSON.
                            """,
                }
            },
            "required"  : ["query"],
        },
    }
]

### Executing SQL queries

Now let's implement the function that will actually excute queries against the database.

In [ ]:
def ask_database(conn, query):
    """Function to query SQLite database with a provided SQL query."""
    try:
        results = str(conn.execute(query).fetchall())
    except Exception as e:
        results = f"query failed with error: {e}"
    return results

def execute_function_call(message):
    if message["function_call"]["name"] == "ask_database":
        query = json.loads(message["function_call"]["arguments"])["query"]
        results = ask_database(conn, query)
    else:
        results = f"Error: function {message['function_call']['name']} does not exist"
    return results

In [ ]:
messages = []
messages.append({"role": "system", "content": "Answer user questions by generating SQL queries against the Chinook Music Database."})
messages.append({"role": "user", "content": "Hi, who are the top 5 artists by number of tracks?"})
chat_response = chat_completion_request(messages, functions)
assistant_message = chat_response.json()["choices"][0]["message"]
messages.append(assistant_message)
if assistant_message.get("function_call"):
    results = execute_function_call(assistant_message)
    messages.append({"role": "function", "name": assistant_message["function_call"]["name"], "content": results})
pretty_print_conversation(messages)

In [ ]:
messages.append({"role": "user", "content": "What is the name of the album with the most tracks?"})
chat_response = chat_completion_request(messages, functions)
assistant_message = chat_response.json()["choices"][0]["message"]
messages.append(assistant_message)
if assistant_message.get("function_call"):
    results = execute_function_call(assistant_message)
    messages.append({"role": "function", "content": results, "name": assistant_message["function_call"]["name"]})
pretty_print_conversation(messages)

## Next Steps

See our other [notebook](How_to_call_functions_for_knowledge_retrieval.ipynb) that demonstrates how to use the Chat Completions API and functions for knowledge retrieval to interact conversationally with a knowledge base.
